In [40]:
#!/usr/bin/env python3
import uproot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import awkward as ak
import time

# Record the start time
start_time = time.perf_counter()
print("Starting analysis...")
HZ4Lep = uproot.open("hz_hww_4l.root")
HZ4Lep_tree = HZ4Lep["Delphes"]
column_arrays = ["MissingET_size","MissingET.Eta","MissingET.Phi","Jet_size","Electron_size","Electron.Eta","Electron.Phi","Muon_size","Muon.Eta","Muon.Phi"]
HZ4Lep_array = HZ4Lep_tree.arrays(column_arrays, library="ak", entry_stop=10)

Starting analysis...


In [46]:
# td=HZ4Lep_array["MissingET.Phi"]-HZ4Lep_array["Muon.Phi"]
ak.firsts(HZ4Lep_array["MissingET.Phi"])


<Array [-2.7, 1.07, -2.81, ... -2.58, 0.914] type='10 * ?float32'>

In [14]:
print(ak.type(HZ4Lep_array))

10 * {"MissingET_size": int32, "MissingET.Eta": var * float32, "MissingET.Phi": var * float32, "Jet_size": int32, "Electron_size": int32, "Electron.Eta": var * float32, "Electron.Phi": var * float32, "Muon_size": int32, "Muon.Eta": var * float32, "Muon.Phi": var * float32}


In [146]:
def check_drFromMET(n_array, name):
    # Calculate delta R between MET and each lepton
    fig=plt.figure()
    ax=fig.add_subplot(111)
    ax.set_title(f"Delta R from MET to Leptons - {name}")
    ax.set_xlabel("Delta R")
    ax.set_ylabel("Count")
    deta_r=np.array([])
    for number_event in range(len(n_array["MissingET_size"])):
        enum=n_array["Electron_size"][number_event]
        mnum=n_array["Muon_size"][number_event]
        if (enum+mnum)!=4:
            continue
        for ie in range(enum):
            dphi=n_array["Electron.Phi"][number_event][ie]-n_array["MissingET.Phi"][number_event]
            while (dphi > np.pi):
                dphi -= 2*np.pi
            while (dphi < -np.pi):
                dphi += 2*np.pi
            dr=np.sqrt((n_array["Electron.Eta"][number_event][ie]-n_array["MissingET.Eta"][number_event])**2 + dphi**2)
            deta_r=np.append(deta_r, dr)
        for im in range(mnum):
            dphi=n_array["Muon.Phi"][number_event][im]-n_array["MissingET.Phi"][number_event]
            while (dphi > np.pi):
                dphi -= 2*np.pi
            while (dphi < -np.pi):
                dphi += 2*np.pi
            dr=np.sqrt((n_array["Muon.Eta"][number_event][im]-n_array["MissingET.Eta"][number_event])**2 + dphi**2)
            deta_r=np.append(deta_r, dr) 
            
    ax.hist(deta_r, bins=100, range=(0, 5))

    plt.savefig(f"DeltaR_MET_Leptons_{name}.png")
    plt.close()
    return 1
check_drFromMET(HZ4Lep_array, "HZ4Lep")

1

In [149]:
HZ4LepLFV = uproot.open("hz_hMuE_LFV.root")

HZ4LepLFV_tree = HZ4LepLFV["Delphes"]
column_arrays = ["MissingET_size","MissingET.Eta","MissingET.Phi","Jet_size","Electron_size","Electron.Eta","Electron.Phi","Muon_size","Muon.Eta","Muon.Phi"]
HZ4LepLFV_array = HZ4LepLFV_tree.arrays(column_arrays, library="np")

In [150]:
check_drFromMET(HZ4LepLFV_array, "HZ4LepLFV")

1